#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbgraos_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  disponibilidade   tipo
0  2017  33959879            33785   soja
1  2018  34777936            33904   soja
2  2019  35895207            31847   soja
3  2020  37191638            32755   soja
4  2021  39126269            34452   soja
5  2022  40894968            29515   soja
6  2017  17427206            56183  milho
7  2018  16126368            51076  milho
8  2019  17515920            57734  milho
9  2020  18258981            56953  milho


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             12 non-null     int64 
 1   producao         12 non-null     int64 
 2   disponibilidade  12 non-null     int64 
 3   tipo             12 non-null     object
dtypes: int64(3), object(1)
memory usage: 512.0+ bytes


### Avaliando a disponibilidade de soja

In [4]:
df.drop(['producao'], axis = 1,inplace=True)
df = df[df['tipo']=='soja']
df.head(10)


,date,disponibilidade,tipo
0,2017,33785,soja
1,2018,33904,soja
2,2019,31847,soja
3,2020,32755,soja
4,2021,34452,soja
5,2022,29515,soja


In [5]:
df.drop(['tipo'], axis = 1,inplace=True)


In [6]:
df.set_index('date',inplace=True)

In [7]:
df=df.sort_index(axis=0)
print(df.head(10))

      disponibilidade
date                 
2017            33785
2018            33904
2019            31847
2020            32755
2021            34452
2022            29515


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [8]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,disponibilidade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(6, 1)"
5,Transformed data shape,"(6, 1)"
6,Transformed train set shape,"(5, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [9]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [10]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [11]:
top4 = exp.compare_models(n_select = 4)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
croston,Croston,6.0245,5.8954,1243.8637,1243.8637,0.0381,0.0375,0.0200
grand_means,Grand Means Forecaster,6.1723,6.0458,1266.8056,1266.8056,0.0386,0.0382,1.9667
naive,Naive Forecaster,6.5904,6.4048,1554.0000,1554.0000,0.0472,0.0471,9.6833
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,7.0153,6.8122,1685.7000,1685.7000,0.0510,0.0510,0.0667
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,7.3037,7.0402,1982.1016,1982.1016,0.0596,0.0603,0.0800
ets,ETS,7.3111,7.0363,2003.0743,2003.0743,0.0606,0.0610,0.0467
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,7.3616,7.0787,2051.6889,2051.6889,0.0619,0.0625,0.2133
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,7.3618,7.0862,2041.8664,2041.8664,0.0614,0.0622,1.9400
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,7.3618,7.0862,2041.8665,2041.8665,0.0614,0.0622,0.2433
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,7.3618,7.0862,2041.8665,2041.8665,0.0614,0.0622,0.0633


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [12]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

## ARIMA model

The ARIMA model is a model based on the Autoregression moving average (ARMA), 
but since the ARMA model is only a model for stationary time series, it is implemented as an ARIMA model including normalizing the abnormal data.

In [13]:
br_cds_dt= exp.create_model('br_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = br_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,18.2857,18.2857,2176.0000,2176.0000,0.0683,0.0661
1,2019.0000,0.7252,0.5415,789.0000,789.0000,0.0241,0.0244
2,2020.0000,3.4281,2.7109,3524.0743,3524.0743,0.1023,0.1078
Mean,nan,7.4797,7.1794,2163.0248,2163.0248,0.0649,0.0661
SD,nan,7.7203,7.9031,1116.6271,1116.6271,0.0320,0.0341


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.



## lightgbm_cds_dt Light Gradient Boosting w/ Cond. Deseasonalize & Detrending 
The series is tested for seasonality using the test outlined in A&N. 
If deemed seasonal, the series is seasonally adjusted using a classical multiplicative decomposition before applying the theta method. The resulting forecasts are then reseasonalised.

In [14]:
lightgbm_cds_dt = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,18.2857,18.2857,2176.0000,2176.0000,0.0683,0.0661
1,2019.0000,1.2252,0.9149,1333.0000,1333.0000,0.0407,0.0415
2,2020.0000,2.5740,2.0355,2646.0667,2646.0667,0.0768,0.0799
Mean,nan,7.3616,7.0787,2051.6889,2051.6889,0.0619,0.0625
SD,nan,7.7441,7.9377,543.2163,543.2163,0.0154,0.0159


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## POLYTREND

In [15]:

polytrend  = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,18.2857,18.2857,2176.0000,2176.0000,0.0683,0.0661
1,2019.0000,1.3919,1.0394,1514.3333,1514.3333,0.0462,0.0473
2,2020.0000,2.5934,2.0508,2666.0000,2666.0000,0.0774,0.0805
Mean,nan,7.4236,7.1253,2118.7778,2118.7778,0.0640,0.0646
SD,nan,7.6963,7.9024,471.9038,471.9038,0.0131,0.0136


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## theta   Theta Forecaster 

In [16]:

theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 9)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:45:17
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,18.2857,18.2857,2175.9996,2175.9996,0.0683,0.0661
1,2019.0000,1.3828,1.0327,1504.5400,1504.5400,0.0459,0.0470
2,2020.0000,2.5944,2.0516,2667.0655,2667.0655,0.0774,0.0805
Mean,nan,7.4210,7.1233,2115.8684,2115.8684,0.0639,0.0645
SD,nan,7.6984,7.9039,476.4999,476.4999,0.0132,0.0137


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,33403.766
2023,33413.016
2024,33422.266
2025,33431.516
2026,33440.766
2027,33450.016
2028,33459.266
2029,33468.516
2030,33477.766


## polytrend

In [17]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})



Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ETS
The ETS algorithm is especially useful for datasets with seasonality and other prior assumptions about the data. 

ETS computes a weighted average over all observations in the input time series dataset as its prediction. 

The weights are exponentially decreasing over time, rather than the constant weights in simple moving average methods. 

The weights are dependent on a constant parameter, which is known as the smoothing parameter.

In [18]:
ets_dt = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ALL GRAPHS

In [19]:
exp.plot_model(estimator = ridge_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, plot = 'forecast',
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
               

NameError: name 'ridge_cds_dt' is not defined

## Avaliando a disponibilidade do milho 

In [20]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbgraos_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  disponibilidade   tipo
0  2017  33959879            33785   soja
1  2018  34777936            33904   soja
2  2019  35895207            31847   soja
3  2020  37191638            32755   soja
4  2021  39126269            34452   soja
5  2022  40894968            29515   soja
6  2017  17427206            56183  milho
7  2018  16126368            51076  milho
8  2019  17515920            57734  milho
9  2020  18258981            56953  milho


In [21]:
df.drop(['producao'], axis = 1,inplace=True)
df = df[df['tipo']=='milho']
df.head(10)

,date,disponibilidade,tipo
6,2017,56183,milho
7,2018,51076,milho
8,2019,57734,milho
9,2020,56953,milho
10,2021,46497,milho
11,2022,52012,milho


In [22]:
df.drop(['tipo'], axis = 1,inplace=True)

In [23]:
df.set_index('date',inplace=True)

In [24]:
df=df.sort_index(axis=0)
print(df.head(10))

      disponibilidade
date                 
2017            56183
2018            51076
2019            57734
2020            56953
2021            46497
2022            52012


In [25]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,disponibilidade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(6, 1)"
5,Transformed data shape,"(6, 1)"
6,Transformed train set shape,"(5, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [26]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [27]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [29]:
top4 = exp.compare_models(n_select = 4)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
croston,Croston,0.9518,0.8450,4208.3843,4208.3843,0.0862,0.0802,0.0167
grand_means,Grand Means Forecaster,1.0952,0.9936,5016.4444,5016.4444,0.0996,0.0950,0.1333
naive,Naive Forecaster,1.3122,1.1948,5965.0000,5965.0000,0.1180,0.1127,0.1600
arima,ARIMA,1.6080,1.5120,8044.0776,8044.0776,0.1515,0.1442,0.1300
polytrend,Polynomial Trend Forecaster,1.6860,1.5601,7800.2778,7800.2778,0.1508,0.1498,0.0433
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.6873,1.5685,7900.2278,7900.2278,0.1517,0.1518,0.1267
theta,Theta Forecaster,1.7156,1.5847,7917.5737,7917.5737,0.1534,0.1518,0.0633
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.7610,1.6331,8224.7667,8224.7667,0.1584,0.1572,0.0367
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.7731,1.6541,8411.8443,8411.8443,0.1606,0.1611,2.0767
exp_smooth,Exponential Smoothing,1.8053,1.6666,8403.1927,8403.1927,0.1629,0.1593,0.0400


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})


### ets ETS 

In [30]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(ets, fh = 9)


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.3037,2.3037,11765.0000,11765.0000,0.2038,0.2269
1,2019.0000,0.0235,0.0233,138.5261,138.5261,0.0024,0.0024
2,2020.0000,3.1474,2.7052,13162.4094,13162.4094,0.2831,0.2480
Mean,nan,1.8249,1.6774,8355.3118,8355.3118,0.1631,0.1591
SD,nan,1.3195,1.1810,5838.0855,5838.0855,0.1181,0.1111


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,41832.6870
2023,37167.7766
2024,32502.8662
2025,27837.9557
2026,23173.0453
2027,18508.1349
2028,13843.2245
2029,9178.3141
2030,4513.4037


### exp_smooth  Exponential Smoothing  

In [31]:
exp_smooth = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.1267,2.1267,10861.1049,10861.1049,0.1881,0.2077
1,2019.0000,0.3489,0.3459,2052.2077,2052.2077,0.0360,0.0367
2,2020.0000,2.9403,2.5272,12296.2656,12296.2656,0.2645,0.2336
Mean,nan,1.8053,1.6666,8403.1927,8403.1927,0.1629,0.1593
SD,nan,1.0821,0.9481,4528.8836,4528.8836,0.0949,0.0873


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ETS

In [ ]:
naive = exp.create_model('naive')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = naive, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,1.0682,1.0682,1389552.0000,1389552.0000,0.0793,0.0826
1,2019.0000,0.5524,0.5521,743061.0000,743061.0000,0.0407,0.0415
2,2020.0000,0.6338,0.6148,725319.0000,725319.0000,0.0382,0.0390
Mean,nan,0.7514,0.7450,952644.0000,952644.0000,0.0527,0.0544
SD,nan,0.2264,0.2299,309025.5059,309025.5059,0.0188,0.0200


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.



## exp_smooth


In [ ]:
croston = exp.create_model('croston')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = croston, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.1682,0.1682,218797.8000,218797.8000,0.0125,0.0126
1,2019.0000,0.6988,0.6984,939979.0200,939979.0200,0.0515,0.0528
2,2020.0000,1.3729,1.3319,1571300.1180,1571300.1180,0.0828,0.0863
Mean,nan,0.7466,0.7328,910025.6460,910025.6460,0.0489,0.0506
SD,nan,0.4930,0.4757,552562.8372,552562.8372,0.0287,0.0302


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### ALL GRAPHS

In [ ]:
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
